In [1]:
# Import data from Excel sheet
import pandas as pd
df = pd.read_excel('hippocampus_volume_relevance_ADNI2.xlsx', sheet_name='ADNI2_LRP_CMP')
sid = df['subject_ID']
grp = df['Group at scan date (1=CN, 2=EMCI, 3=LMCI, 4=AD, 5=SMC)']
age = df['Age at scan']
sex = df['Sex (1=female)']
tiv = df['TIV_CAT12']
field = df['MRI_Field_Strength']
amybin = df['Amy SUVR Pos']
grpbin = (grp > 1) # 1=CN, ...

In [2]:
# Scan for nifti file names
import glob
dataAD = sorted(glob.glob('mwp1_MNI/AD/*.nii.gz'))
dataLMCI = sorted(glob.glob('mwp1_MNI/LMCI/*.nii.gz'))
dataCN = sorted(glob.glob('mwp1_MNI/CN/*.nii.gz'))
dataFiles = dataAD + dataLMCI + dataCN
numfiles = len(dataFiles)
print('Found ', str(numfiles), ' nifti files')

Found  663  nifti files


In [3]:
import re
debug = False
cov_idx = [-1] * numfiles # list; array: np.full((numfiles, 1), -1, dtype=int)
print('Matching covariates for loaded files ...')
for i,id in enumerate(sid):
    p = [j for j,x in enumerate(dataFiles) if re.search('_%04d_' % id, x)] # translate ID numbers to four-digit numbers, get both index and filename
    if len(p)==0:
        if debug: print('Did not find %04d' % id) # did not find Excel sheet subject ID in loaded file selection
    else:
        if debug: print('Found %04d in %s: %s' % (id, p[0], dataFiles[p[0]]))
        cov_idx[p[0]] = i # store Excel index i for data file index p[0]
print('Checking for scans not found in Excel sheet: ', sum(x<0 for x in cov_idx))

labels = pd.DataFrame({'Group':grpbin}).iloc[cov_idx, :]
grps = pd.DataFrame({'Group':grp, 'RID':sid}).iloc[cov_idx, :]

Matching covariates for loaded files ...
Checking for scans not found in Excel sheet:  0


In [4]:
#Load residualized data from disk
import h5py
import numpy as np
hf = h5py.File('residuals_ADNI2_wb_mwp1_CAT12_MNI.hdf5', 'r')
hf.keys # read keys
images = np.array(hf.get('images'))
hf.close()

In [5]:
import tensorflow as tf
print(tf.__version__)
# disable tensorflow deprecation warnings
import logging
logging.getLogger('tensorflow').disabled=True

1.15.4


In [6]:
import nibabel as nib

# define FOV to reduce required memory size
x_range_from = 10; x_range_to = 110
y_range_from = 13; y_range_to = 133
z_range_from = 5; z_range_to = 105

fname_overlay = sorted(glob.glob('Hippocampus_masks/aal_*.nii*')) + glob.glob('Hippocampus_masks/brainmask.nii*')
data_overlay = np.zeros((len(fname_overlay), z_range_to-z_range_from, x_range_to-x_range_from, y_range_to-y_range_from, 1), dtype=np.float32) # numfiles× z × x × y ×1; avoid 64bit types

for i in range(len(fname_overlay)):   
    img = nib.load(fname_overlay[i])
    img = img.get_fdata()[x_range_from:x_range_to, y_range_from:y_range_to, z_range_from:z_range_to]
    img = np.transpose(img, (2, 0, 1)) # reorder dimensions to match coronal view z*x*y in MRIcron etc.
    img = np.flip(img) # flip all positions
    data_overlay[i, :,:,:, 0] = np.nan_to_num(img)

In [7]:
###### check which files were loaded #####
print(fname_overlay)
print(data_overlay.shape)

['Hippocampus_masks\\aal_basal_ganglia.nii.gz', 'Hippocampus_masks\\aal_cerebellum.nii.gz', 'Hippocampus_masks\\aal_frontal.nii.gz', 'Hippocampus_masks\\aal_hippocampus.nii.gz', 'Hippocampus_masks\\aal_insula_cingulate.nii.gz', 'Hippocampus_masks\\aal_occipital.nii.gz', 'Hippocampus_masks\\aal_parietal.nii.gz', 'Hippocampus_masks\\aal_temporal.nii.gz', 'Hippocampus_masks\\brainmask.nii.gz']
(9, 100, 100, 120, 1)


In [8]:
# initialize parameters for analyzer
from keras.models import load_model, Model
#!pip install innvestigate
import innvestigate
import innvestigate.utils as iutils
import numpy as np
from matplotlib import pyplot as plt
import scipy
import csv
import os

# see https://github.com/albermax/innvestigate/blob/master/examples/notebooks/imagenet_compare_methods.ipynb for a list of alternative methods
methods = [ # tuple with method,     params,                  label
#            ("deconvnet",            {},                      "Deconvnet"),
#            ("guided_backprop",      {},                      "Guided Backprop"),
#            ("deep_taylor.bounded",  {"low": -1, "high": 1},  "DeepTaylor"),
#            ("input_t_gradient",     {},                      "Input * Gradient"),
#            ("lrp.z",                {},                      "LRP-Z"),
#            ("lrp.epsilon",          {"epsilon": 1},          "LRP-epsilon"),
#            ("lrp.alpha_1_beta_0",   {"neuron_selection_mode":"index"},     "LRP-alpha1beta0"),
    ("lrp.sequential_preset_a", {"neuron_selection_mode": "index", "epsilon": 1e-10}, "LRP-CMPalpha1beta0"), # LRP CMP rule taken from https://github.com/berleon/when-explanations-lie/blob/master/when_explanations_lie.py
]

# define output csv header manually
header = ['SID', 'sum_relevance_basal_ganglia', 'sum_relevance_cerebellum', 'sum_relevance_frontal', 
          'sum_relevance_hippocampus', 'sum_relevance_insula_cingulate', 'sum_relevance_occipital', 
          'sum_relevance_parietal', 'sum_relevance_temporal', 'sum_relevance_brain']

Using TensorFlow backend.


In [9]:
# Load CNN model from disk and create analyzer

for k in range(10):
    mymodel = load_model('model_checkpoints/resmodel_wb_cv%d.best.hdf5' % (k+1))
    #mymodel.summary()
    #model_wo_softmax = iutils.keras.graph.model_wo_softmax(mymodel)  ## sometimes raises: ValueError: The name "dense_1" is used 2 times in the model. All layer names should be unique.
    #model_wo_softmax = Model(inputs=mymodel.inputs,
    #                          outputs=iutils.keras.graph.pre_softmax_tensors(mymodel.outputs),
    #                          name=('wo_softmax_cv%d' % (k+1))) 
    #
    mymodel.layers[-1].activation=tf.keras.activations.linear
    mymodel.save('tmp_wo_softmax.hdf5')
    model_wo_softmax = load_model('tmp_wo_softmax.hdf5')
    os.remove('tmp_wo_softmax.hdf5')
    if (k==0):
        model_wo_softmax.summary()

    # create analyzer
    analyzers = []
    for method in methods:
        analyzer = innvestigate.create_analyzer(method[0], model_wo_softmax, **method[1])
        # Some analyzers require training.
        #   analyzer.fit(test_img, batch_size=30, verbose=1)
        #  analyzers.append(analyzer)

    with open(('hipp_lrpCMP_resmodel_ADNI2_cv%d.csv' % (k+1)), 'w', newline='') as csvfile:
        csvwriter = csv.writer(csvfile, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        csvwriter.writerow(header)
        print('exporting values to hipp_lrpCMP_resmodel_ADNI2_cv%d.csv' % (k+1))
        
        for indx in range(len(grps)): # for all scans
            test_img = images[indx]
            test_img = np.reshape(test_img, (1,)+ test_img.shape) # add first subj index again to mimic original array structure

            #for method,analyzer in zip(methods, analyzers):
            a = analyzer.analyze(test_img, neuron_selection=1)
            np.clip(a, a_min=0, a_max=None, out=a) # set negative values to 0
            #a = np.squeeze(a)
            #a = scipy.ndimage.filters.gaussian_filter(a, sigma=0.8) # smooth activity image
            #scale = np.quantile(a, 0.99) # no need for abs(a)
            #if scale!=0:  # ignore if relevance maps contains only zeros, output will be zero as well
            #    a = (a/scale)

            rel_all = np.multiply(data_overlay, np.repeat(a, len(fname_overlay), axis=0))

            csvwriter.writerow([grps.RID.to_numpy(dtype=np.int)[indx], ] + np.sum(rel_all, axis=(1,2,3,4)).tolist())
        csvfile.close()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_1 (Conv3D)            (None, 100, 100, 120, 5)  140       
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 50, 50, 60, 5)     0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 50, 50, 60, 5)     20        
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 50, 50, 60, 5)     680       
_________________________________________________________________
max_pooling3d_2 (MaxPooling3 (None, 25, 25, 30, 5)     0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 25, 25, 30, 5)     20        
_________________________________________________________________
conv3d_3 (Conv3D)            (None, 25, 25, 30, 5)     680       
__________

In [10]:
# run again for whole-dataset model
for k in [0]:
    mymodel = load_model('model_checkpoints/resmodel_wb_whole_ds.hdf5')
    #mymodel.summary()
    #model_wo_softmax = iutils.keras.graph.model_wo_softmax(mymodel)  ## sometimes raises: ValueError: The name "dense_1" is used 2 times in the model. All layer names should be unique.
    #model_wo_softmax = Model(inputs=mymodel.inputs,
    #                          outputs=iutils.keras.graph.pre_softmax_tensors(mymodel.outputs),
    #                          name=('wo_softmax_cv%d' % (k+1))) 
    #
    mymodel.layers[-1].activation=tf.keras.activations.linear
    mymodel.save('tmp_wo_softmax.hdf5')
    model_wo_softmax = load_model('tmp_wo_softmax.hdf5')
    os.remove('tmp_wo_softmax.hdf5')
    if (k==0):
        model_wo_softmax.summary()

    # create analyzer
    analyzers = []
    for method in methods:
        analyzer = innvestigate.create_analyzer(method[0], model_wo_softmax, **method[1])
        # Some analyzers require training.
        #   analyzer.fit(test_img, batch_size=30, verbose=1)
        #  analyzers.append(analyzer)

    with open(('hipp_lrpCMP_resmodel_ADNI2_whole_ds.csv'), 'w', newline='') as csvfile:
        csvwriter = csv.writer(csvfile, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        csvwriter.writerow(header)
        if (k==0):
            print('hipp_lrpCMP_resmodel_ADNI2_whole_ds.csv')
            print(header)

        for indx in range(len(grps)): # for all scans
            test_img = images[indx]
            test_img = np.reshape(test_img, (1,)+ test_img.shape) # add first subj index again to mimic original array structure

            #for method,analyzer in zip(methods, analyzers):
            a = analyzer.analyze(test_img, neuron_selection=1)
            np.clip(a, a_min=0, a_max=None, out=a) # set negative values to 0
            #a = np.squeeze(a)
            #a = scipy.ndimage.filters.gaussian_filter(a, sigma=0.8) # smooth activity image
            #scale = np.quantile(a, 0.99) # no need for abs(a)
            #if scale!=0:  # ignore if relevance maps contains only zeros, output will be zero as well
            #    a = (a/scale)

            rel_all = np.multiply(data_overlay, np.repeat(a, len(fname_overlay), axis=0))

            print([grps.RID.to_numpy(dtype=np.int)[indx], ] + np.sum(rel_all, axis=(1,2,3,4)).tolist())
            csvwriter.writerow([grps.RID.to_numpy(dtype=np.int)[indx], ] + np.sum(rel_all, axis=(1,2,3,4)).tolist())
        csvfile.close()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_1 (Conv3D)            (None, 100, 100, 120, 5)  140       
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 50, 50, 60, 5)     0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 50, 50, 60, 5)     20        
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 50, 50, 60, 5)     680       
_________________________________________________________________
max_pooling3d_2 (MaxPooling3 (None, 25, 25, 30, 5)     0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 25, 25, 30, 5)     20        
_________________________________________________________________
conv3d_3 (Conv3D)            (None, 25, 25, 30, 5)     680       
__________

[5120, 0.17036455869674683, 1.5768071413040161, 2.6102843284606934, 0.14239946007728577, 0.4812507629394531, 2.655569314956665, 1.7444021701812744, 3.77461838722229, 14.308283805847168]
[5241, 0.5116649270057678, 1.2296204566955566, 2.4851651191711426, 2.3725521564483643, 0.6655858159065247, 2.108698606491089, 1.8689956665039062, 9.937889099121094, 20.754514694213867]
[5054, 0.2192753702402115, 0.5264725089073181, 1.8984793424606323, 0.7771696448326111, 0.5279770493507385, 0.9925023317337036, 1.1063919067382812, 3.4709906578063965, 9.59462833404541]
[408, 1.0737814903259277, 0.7416065335273743, 2.535738945007324, 2.2364320755004883, 0.6827662587165833, 1.2755987644195557, 1.7388900518417358, 11.379083633422852, 21.954805374145508]
[835, 0.7446225881576538, 0.9041056632995605, 3.0617432594299316, 3.7211544513702393, 0.6946383714675903, 2.6002488136291504, 1.4585680961608887, 10.058332443237305, 21.465600967407227]
[4964, 0.1793108582496643, 0.5361945629119873, 1.7390457391738892, 0.8324

[6039, 0.5263073444366455, 0.9794266819953918, 4.422682762145996, 0.7977911829948425, 1.6938354969024658, 4.414473056793213, 4.948263645172119, 14.396732330322266, 33.80207061767578]
[361, 0.6244103312492371, 0.9370681047439575, 2.5257365703582764, 0.9230058193206787, 0.46445873379707336, 1.3176530599594116, 1.8505662679672241, 8.241374015808105, 17.599767684936523]
[1427, 0.15877044200897217, 0.9832024574279785, 2.699965238571167, 0.05967117100954056, 0.6209198236465454, 2.4867775440216064, 1.6298555135726929, 1.87327241897583, 11.49919319152832]
[4992, 0.9370551109313965, 1.7580569982528687, 3.221487522125244, 4.201399326324463, 1.635957956314087, 5.147395133972168, 5.2040019035339355, 24.974470138549805, 45.94788360595703]
[5028, 0.8303741812705994, 0.8895829319953918, 3.9002158641815186, 0.5226815938949585, 1.198976993560791, 1.1402376890182495, 3.0379929542541504, 8.538911819458008, 21.184749603271484]
[5056, 0.14512744545936584, 0.6286175847053528, 1.914583444595337, 0.9756268262

[4252, 0.48027902841567993, 0.5681796073913574, 1.5802578926086426, 2.153618097305298, 0.28280550241470337, 1.7629101276397705, 1.1265813112258911, 6.720617294311523, 13.722691535949707]
[4258, 0.6584477424621582, 1.080165982246399, 3.8027358055114746, 1.3958566188812256, 1.2143663167953491, 3.0202810764312744, 2.0265872478485107, 6.671140193939209, 21.731788635253906]
[4280, 1.0773972272872925, 0.9937270283699036, 2.7995994091033936, 2.162750005722046, 0.6044079065322876, 1.167732834815979, 1.1155966520309448, 9.524581909179688, 18.947031021118164]
[4282, 0.2732030153274536, 0.9320587515830994, 2.6098320484161377, 0.9417546987533569, 0.44398728013038635, 2.3922231197357178, 1.505401372909546, 5.265550136566162, 14.989225387573242]
[4307, 0.7809791564941406, 1.2116942405700684, 3.6992931365966797, 3.458585023880005, 0.7173682451248169, 2.276794910430908, 2.2237184047698975, 16.305736541748047, 29.87091636657715]
[4338, 1.0105400085449219, 1.211690068244934, 2.5543034076690674, 3.038872

[4802, 0.2654121518135071, 0.6403934955596924, 1.9281063079833984, 0.3489977717399597, 0.4186054766178131, 0.8379226326942444, 1.631754755973816, 6.841955661773682, 13.471957206726074]
[4820, 0.6237610578536987, 0.589414119720459, 2.6743152141571045, 1.1380972862243652, 0.46846237778663635, 1.0186060667037964, 3.709296226501465, 9.537818908691406, 20.792543411254883]
[4827, 0.4250880181789398, 0.6199374794960022, 2.645258903503418, 0.42020130157470703, 0.48935991525650024, 1.6188836097717285, 1.9952664375305176, 3.002688407897949, 11.783692359924316]
[4845, 1.3186677694320679, 0.8661859035491943, 4.297879695892334, 2.6509108543395996, 1.2978291511535645, 2.233768939971924, 5.282388210296631, 15.742483139038086, 33.76708984375]
[4853, 0.8174890279769897, 1.3562389612197876, 3.03462815284729, 0.790681004524231, 0.9050570726394653, 3.17160701751709, 2.3492965698242188, 6.887150287628174, 20.948970794677734]
[4863, 0.13471876084804535, 0.535816490650177, 1.8649879693984985, 0.4590530395507

...

[722, 0.2225162535905838, 1.2378381490707397, 2.026071071624756, 0.3719028830528259, 0.2951045632362366, 2.047128200531006, 1.4413076639175415, 2.277080535888672, 10.550895690917969]
[767, 0.17804789543151855, 0.774135947227478, 2.523559808731079, 0.043134041130542755, 0.30240556597709656, 2.1623380184173584, 1.5579956769943237, 1.2841392755508423, 9.605097770690918]
[6051, 0.3484046757221222, 0.548903226852417, 2.2940120697021484, 0.01895029842853546, 0.48113447427749634, 0.7765456438064575, 1.6866265535354614, 1.7153209447860718, 8.775191307067871]
[4255, 0.24781827628612518, 0.7231884002685547, 3.121166467666626, 0.46415069699287415, 0.6344919204711914, 1.338809609413147, 1.7144919633865356, 3.38600754737854, 12.419480323791504]
[21, 0.16314324736595154, 0.8429071307182312, 1.8120148181915283, 0.0263485386967659, 0.4756496250629425, 0.6809500455856323, 1.0214930772781372, 1.373509168624878, 6.978289604187012]
[89, 0.09789888560771942, 0.5060785412788391, 2.552398920059204, 0.0170084

[4148, 0.23966340720653534, 0.6910870671272278, 2.966637372970581, 0.07879983633756638, 0.9094889163970947, 2.714599847793579, 2.285644769668579, 1.3969067335128784, 12.240503311157227]
[4150, 0.2752470374107361, 1.2328767776489258, 3.837920904159546, 0.130815327167511, 0.6064448356628418, 2.747382402420044, 2.341707229614258, 4.601572513580322, 17.676420211791992]
[4151, 0.24214614927768707, 0.8321817517280579, 1.6015928983688354, 0.23257790505886078, 0.32900553941726685, 0.7417113780975342, 0.9346789717674255, 3.806534767150879, 9.897990226745605]
[4155, 0.2018895298242569, 0.7597872018814087, 1.5563125610351562, 0.021014802157878876, 0.3895391821861267, 0.9002848267555237, 0.736991822719574, 0.9784557819366455, 6.14904260635376]
[4158, 0.10040237754583359, 0.7689167261123657, 2.400463581085205, 0.2188558578491211, 0.4954354465007782, 0.7335563898086548, 1.4881401062011719, 2.2667620182037354, 9.114326477050781]
[4164, 0.26399585604667664, 1.4473631381988525, 3.409917116165161, 0.111

[4367, 0.19936560094356537, 0.5126162767410278, 1.9165623188018799, 0.03888966888189316, 0.45752015709877014, 1.3577042818069458, 0.7420504093170166, 1.5571433305740356, 7.455252170562744]
[4369, 0.07176943868398666, 0.5686945915222168, 1.5943635702133179, 0.009419552981853485, 0.30260467529296875, 1.2378212213516235, 0.7431171536445618, 0.5322508811950684, 5.467029094696045]
[4371, 0.28013625741004944, 0.8195834755897522, 2.0786826610565186, 0.43174782395362854, 0.4065743684768677, 1.7552568912506104, 1.4668484926223755, 2.7167422771453857, 10.74509334564209]
[4372, 0.08193452656269073, 0.6064298748970032, 1.651033878326416, 0.004291553981602192, 0.1775178462266922, 1.0583170652389526, 1.2935669422149658, 0.6401172876358032, 5.962079048156738]
[4376, 0.4385439455509186, 0.35996949672698975, 1.1949228048324585, 0.18827584385871887, 0.24797256290912628, 0.705230176448822, 0.7346948385238647, 1.0333499908447266, 5.589439868927002]
[4382, 0.1510821431875229, 0.7096783518791199, 2.25769019

[4508, 0.0909041315317154, 1.1186619997024536, 2.5219171047210693, 0.014641444198787212, 0.32628342509269714, 1.8422991037368774, 2.836047649383545, 1.4190574884414673, 11.070794105529785]
[4516, 0.05365565046668053, 0.6251415014266968, 2.060417413711548, 0.03365359082818031, 0.33564773201942444, 1.2931010723114014, 1.7130331993103027, 1.4649658203125, 8.2206392288208]
[4520, 0.18494409322738647, 0.7230820059776306, 1.8545421361923218, 0.053450386971235275, 0.364489883184433, 2.068410873413086, 1.7988872528076172, 1.4337314367294312, 9.31272029876709]
[4545, 0.5509650707244873, 1.0334112644195557, 3.091672658920288, 0.1772540807723999, 0.8072561025619507, 2.155454397201538, 2.381343126296997, 2.174474000930786, 13.63994026184082]
[4552, 0.5211228132247925, 0.786230742931366, 2.484239101409912, 1.0962016582489014, 0.28892719745635986, 1.5546855926513672, 1.2761728763580322, 4.534966945648193, 12.830315589904785]
[4555, 0.16519282758235931, 0.5609574317932129, 1.9898011684417725, 0.05533

[4921, 0.730634331703186, 1.0400846004486084, 2.5205447673797607, 1.0145288705825806, 0.718061625957489, 2.0325303077697754, 2.2492027282714844, 12.262651443481445, 24.141143798828125]
[4951, 0.22752253711223602, 0.56966233253479, 1.9662048816680908, 0.019709117710590363, 0.3168720602989197, 1.1133081912994385, 1.464748740196228, 1.2589586973190308, 7.639847278594971]
[4952, 0.10710509121417999, 0.8999736309051514, 3.33921480178833, 0.06090978905558586, 0.33352139592170715, 2.081972122192383, 1.58224618434906, 1.7444133758544922, 11.533225059509277]
